In [1]:
import os
import sys
import comtypes.client

In [2]:
# False,表示脚本将启动一个新的sap2000实例，而不是连接已有的。-- False方案好像已失效，以后自动手动打开软件后，用True方案。
# AttachToInstance = False
AttachToInstance = True

In [3]:
# True则可以手动指定sap2000的安装路径，False则不会执行安装路径的代码，而会自动打开最新安装的sap2000版本。
SpecifyPath = True
ProgramPath = r'C:\Program Files\Computers and Structures\SAP2000 22\SAP2000.exe'

In [4]:
# 设置模型文件的保存路径。
APIPath = r'C:\Users\Administrator\Desktop\SAP'
if not os.path.exists(APIPath): # 如果不存在，则创建
    try:
        os.makedirs(APIPath)
    except OSError:
        pass
# 拼接，os.sep: 操作系统的分隔符（在windows下是 \
ModelPath = APIPath + os.sep + 'API_1-001.sdb'

In [5]:

if AttachToInstance:
    #attach to a running instance of SAP2000
    # 连接到已打开SAP2000实例
    try:
        #get the active SapObject
        # 尝试获取当前正在运行的SAP2000实例的对象，"CSI.SAP2000.API.SapObject"是SAP2000在系统注册表中的COM接口名。
        mySapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
    except (OSError, comtypes.COMError):
        print("No running instance of the program found or failed to attach.")
        sys.exit(-1)
else:
    #create API helper object
    # 创建一个新的实例
    # 'SAP2000v22.Helper' SAP2000提供的COM辅助类，用来启动主程序、创建对象等。
    helper = comtypes.client.CreateObject('SAP2000v22.Helper')
    # 调用QueryInterface将对象转换为具体的cHelper接口，这样才能使用其中定义的方法。
    helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
    if SpecifyPath:
        try:
            #'create an instance of the SAPObject from the specified path
            mySapObject = helper.CreateObject(ProgramPath)
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program from " + ProgramPath)
            sys.exit(-1)
    else:
        try:
            #create an instance of the SAPObject from the latest installed SAP2000
            mySapObject = helper.CreateObjectProgID("CSI.SAP2000.API.SapObject")
        except (OSError, comtypes.COMError):
            print("Cannot start a new instance of the program.")
            sys.exit(-1)
        #start SAP2000 application
        mySapObject.ApplicationStart()



In [6]:
#create SapModel object
# 从SAP2000主对象中提取出SapModel，这是所有结构建模操作的核心对象，之后定义节点、杆件、荷载等操作都通过它来进行。
SapModel = mySapObject.SapModel


In [7]:
#initialize model
SapModel.InitializeNewModel()

0

In [8]:
#create new blank model
ret = SapModel.File.NewBlank()

In [9]:
#define material property
# 2是SAP2000中混凝土的材料类型编号
MATERIAL_CONCRETE = 2
ret = SapModel.PropMaterial.SetMaterial('CONC', MATERIAL_CONCRETE)
    #assign isotropic mechanical properties to material
    # 泊松比，不能超过0.5
ret = SapModel.PropMaterial.SetMPIsotropic('CONC', 3600, 0.2, 0.0000055)

In [10]:
#define rectangular frame section property
# SetRectangle 设置矩形截面 R1:横截面名 12, 12:几何尺寸，高宽 
ret = SapModel.PropFrame.SetRectangle('R1', 'CONC', 12, 12)
    #define frame section property modifiers
# 修正系数
ModValue = [1000, 0, 0, 1, 1, 1, 1, 1]
ret = SapModel.PropFrame.SetModifiers('R1', ModValue)


In [11]:
#switch to k-ft units
kip_ft_F = 4
ret = SapModel.SetPresentUnits(kip_ft_F)

In [12]:
#add frame object by coordinates
FrameName1 = ' '
FrameName2 = ' '
FrameName3 = ' '
[FrameName1, ret] = SapModel.FrameObj.AddByCoord(0, 0, 0, 0, 0, 10, FrameName1, 'R1', '1', 'Global')
[FrameName2, ret] = SapModel.FrameObj.AddByCoord(0, 0, 10, 8, 0, 16, FrameName2, 'R1', '2', 'Global')
[FrameName3, ret] = SapModel.FrameObj.AddByCoord(-4, 0, 10, 0, 0, 10, FrameName3, 'R1', '3', 'Global')

In [13]:
#assign point object restraint at base
PointName1 = ' '
PointName2 = ' '
Restraint = [True, True, True, True, False, False]
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName1, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName1, Restraint)

In [14]:
#assign point object restraint at top
Restraint = [True, True, False, False, False, False]
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)
ret = SapModel.PointObj.SetRestraint(PointName2, Restraint)

In [15]:
#refresh view, update (initialize) zoom
ret = SapModel.View.RefreshView(0, False)

In [16]:
#add load patterns

LTYPE_OTHER = 8
ret = SapModel.LoadPatterns.Add('1', LTYPE_OTHER, 1, True)
ret = SapModel.LoadPatterns.Add('2', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('3', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('4', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('5', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('6', LTYPE_OTHER, 0, True)
ret = SapModel.LoadPatterns.Add('7', LTYPE_OTHER, 0, True)

In [17]:
#assign loading for load pattern 2

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
PointLoadValue = [0,0,-10,0,0,0]
ret = SapModel.PointObj.SetLoadForce(PointName1, '2', PointLoadValue)
ret = SapModel.FrameObj.SetLoadDistributed(FrameName3, '2', 1, 10, 0, 1, 1.8, 1.8)


In [18]:
#assign loading for load pattern 3

[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName3, PointName1, PointName2)
PointLoadValue = [0,0,-17.2,0,-54.4,0]
ret = SapModel.PointObj.SetLoadForce(PointName2, '3', PointLoadValue)

In [19]:
#assign loading for load pattern 4

ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '4', 1, 11, 0, 1, 2, 2)
#assign loading for load pattern 5
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '5', 1, 2, 0, 1, 2, 2, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '5', 1, 2, 0, 1, -2, -2, 'Local')
#assign loading for load pattern 6
ret = SapModel.FrameObj.SetLoadDistributed(FrameName1, '6', 1, 2, 0, 1, 0.9984, 0.3744, 'Local')
ret = SapModel.FrameObj.SetLoadDistributed(FrameName2, '6', 1, 2, 0, 1, -0.3744, 0, 'Local')
#assign loading for load pattern 7
ret = SapModel.FrameObj.SetLoadPoint(FrameName2, '7', 1, 2, 0.5, -15, 'Local')

In [20]:
#switch to k-in units

kip_in_F = 3
ret = SapModel.SetPresentUnits(kip_in_F)

In [21]:
#save model
ret = SapModel.File.Save(ModelPath)

In [22]:
#run model (this will create the analysis model)
ret = SapModel.Analyze.RunAnalysis()

In [23]:
#initialize for Sap2000 results
SapResult= [0,0,0,0,0,0,0]
[PointName1, PointName2, ret] = SapModel.FrameObj.GetPoints(FrameName2, PointName1, PointName2)

In [24]:
#get Sap2000 results for load cases 1 through 7

for i in range(0,7):
      NumberResults = 0
      Obj = []
      Elm = []
      ACase = []
      StepType = []
      StepNum = []
      U1 = []
      U2 = []
      U3 = []
      R1 = []
      R2 = []
      R3 = []
      ObjectElm = 0;
      ret = SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()
      ret = SapModel.Results.Setup.SetCaseSelectedForOutput(str(i+1))
      if i <= 3:
          [NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret] = SapModel.Results.JointDispl(PointName2, ObjectElm, NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3)
          SapResult[i] = U3[0]
      else:
          [NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret] = SapModel.Results.JointDispl(PointName1, ObjectElm, NumberResults, Obj, Elm, ACase, StepType, StepNum, U1, U2, U3, R1, R2, R3)
          SapResult[i] = U1[0]

In [25]:
#close Sap2000

ret = mySapObject.ApplicationExit(False)
SapModel = None
mySapObject = None

In [26]:
#fill independent results

IndResult= [0,0,0,0,0,0,0]
IndResult[0] = -0.02639
IndResult[1] = 0.06296
IndResult[2] = 0.06296
IndResult[3] = -0.2963
IndResult[4] = 0.3125
IndResult[5] = 0.11556
IndResult[6] = 0.00651

In [27]:
#fill percent difference

PercentDiff = [0,0,0,0,0,0,0]
for i in range(0,7):
      PercentDiff[i] = (SapResult[i] / IndResult[i]) - 1